In [ ]:
import cv2
import time
import numpy as np
import math
import pyautogui

pyautogui.FAILSAFE = False
SCREEN_X, SCREEN_Y = pyautogui.size()
MOVEMENT_START = None
CLICK = None
LEFT = None
PRE = None

cap = cv2.VideoCapture(0)

# While is used to display the images continously as a video...
while(1):
    
    check,imginv = cap.read() 
    img = cv2.flip(imginv, 1)
    
    
    
    #cv2.rectangle(img, (700,300), (400,100), (0,255,0),0)
    crop_img = img[100:300,450:700]
    

    
    
    CAMERA_X, CAMERA_Y, channels = crop_img.shape

    
    
    # Display the image
    cv2.imshow('Video', crop_img)
    
    # Our operations on the frame come here
    grey = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
    
    # hsv image
#     hsv = cv2.cvtColor(crop_img, cv2.COLOR_BGR2HSV)
    
    cv2.imshow('Grey',grey)
    
    # Applying Gaussian Blur
    #value = (35, 35)
    blurred = cv2.GaussianBlur(grey, (35, 35), 0)
    cv2.imshow('Blured',blurred)
    
    # Thresholded image
    _, thresh1 = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    cv2.imshow('Thresholded', thresh1)
    
     
    image, contours, hierarchy = cv2.findContours(thresh1.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    
    
    # Finding the contour with max area( Biggest Contour )
    max_area = -1
    for i in range(len(contours)):
        cnt = contours[i]
        area = cv2.contourArea(cnt)
        if area > max_area:
            max_area = area
            ci = i
    cnt = contours[ci]
    
#   print(cnt)
    x, y, w, h = cv2.boundingRect(cnt)
    
#   print(x,y,w,h)
    cv2.rectangle(crop_img, (x, y), (x + w, y + h), (0, 0, 255), 0)
    
#    cv2.imshow('Gesture', crop_img)
    
    #finding convex hull
    hull = cv2.convexHull(cnt)
    
    # Drawing contours
    drawing = np.zeros(crop_img.shape,np.uint8)
    cv2.drawContours(drawing, [cnt], 0, (0, 255, 0), 0)
    cv2.drawContours(drawing, [hull], 0,(0, 0, 255), 0)
    
    hull = cv2.convexHull(cnt, returnPoints=False)
    defects = cv2.convexityDefects(cnt, hull)
    
#   print(defects[0,0])
#    cv2.drawContours(thresh1, contours, -1, (0, 255, 0), 10)
    
   
    cv2.imshow('Drawing', drawing)
    
    
    count_defects = 0
    used_defect = None
    for i in range(defects.shape[0]):
        s, e, f, d = defects[i, 0]
        start = tuple(cnt[s][0])
        end = tuple(cnt[e][0])
        far = tuple(cnt[f][0])
        a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
        b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
        c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
        angle = math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 57
        
        if angle <= 90:
            count_defects += 1
            
        cv2.circle(crop_img, far, 5, [0, 0, 255], -1)
        cv2.line(crop_img, start, end, [0, 255, 0], 2)
        

        if count_defects == 1 and angle <= 90:
            used_defect = {"x": start[0], "y": start[1]}
            
        if count_defects == 3 and angle <= 90:
            point = (start[0], start[1])
            
            
    
    cv2.putText(crop_img, str(count_defects+1), (60, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    
    
## Pressing Space Bar...   

    if count_defects == 2:
            LEFT=time.time()
            pyautogui.press('space')

    
    cv2.imshow('cropimage', crop_img)
    
    cv2.imshow('Drawing', drawing)
    
    
    key=cv2.waitKey(1)
    if key==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()